In [12]:
from sklearn.datasets import load_breast_cancer

In [13]:
import pandas as pd

In [14]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
y = pd.DataFrame(data.target, columns=['target'])
y.head()

,target
0,0
1,0
2,0
3,0
4,0


In [16]:
X.shape, y.shape

((569, 30), (569, 1))

In [17]:
y.value_counts()

target
1         357
0         212
Name: count, dtype: int64

In [18]:
# preprocessing the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
# convert pytorch te
import torch
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [21]:
# define the model
import torch.nn as nn
class BreastCancerModel(nn.Module):
  def __init__(self):
    super(BreastCancerModel,self).__init__()
    self.fc1 = nn.Linear(30,16)
    self.fc2 = nn.Linear(16,8)
    self.fc3 = nn.Linear(8,1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.sigmoid(x)
    return x


In [22]:
model = BreastCancerModel()

In [23]:
criterian = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [24]:
epochs = 100
for epoch in range(epochs):
  model.train()
  output = model(x_train_tensor)
  loss = criterian(output,y_train_tensor)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch+ 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{epochs}], loss:{loss.item():.4f}')


Epoch [10/100], loss:0.6592
Epoch [20/100], loss:0.6228
Epoch [30/100], loss:0.5819
Epoch [40/100], loss:0.5345
Epoch [50/100], loss:0.4807
Epoch [60/100], loss:0.4223
Epoch [70/100], loss:0.3634
Epoch [80/100], loss:0.3084
Epoch [90/100], loss:0.2608
Epoch [100/100], loss:0.2218


In [25]:
# evaluate the model
with torch.no_grad():
  model.eval()
  output = model(x_test_tensor)
  preds = (output > 0.5).float()
  accuracy = (preds == y_test_tensor).sum()/len(y_test_tensor)
  print(f"Test Accuracy : {accuracy:.4f}")

Test Accuracy : 0.9737


In [26]:
sample = X_test[0].reshape(1, -1)
sample_tensor = torch.tensor(sample, dtype=torch.float32)
with torch.no_grad():
    prob = model(sample_tensor).item()
    print("Probability benign:", prob)
    print("Prediction:", 1 if prob > 0.5 else 0)


Probability benign: 0.7095881700515747
Prediction: 1


In [27]:
torch.save(model.state_dict(), "breast_cancer_model.pth")

In [28]:
model_loaded = BreastCancerModel()
model_loaded.load_state_dict(torch.load("breast_cancer_model.pth", weights_only=True))
model_loaded.eval()


BreastCancerModel(
  (fc1): Linear(in_features=30, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)